In [1]:
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
data_dir = 'drive/My Drive/'
train_lang = 'en'

In [4]:
class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):
	#open file
        with open(data_dir + train_lang + '.train', 'r') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        self.target_vocab = {} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]

	    #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        for line in train:
            sequence = []
            target = []
            chars = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    for char in word:
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], #  (1)
        }

In [5]:
dataset = DatasetSeq(data_dir)

In [37]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    # pad different length sequences
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

In [38]:
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes, class_model):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.class_model = class_model(emb_dim, hidden_dim, batch_first=True)
        
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Ebm_dim
        hidden, _ = self.class_model(emb) # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes
        return pred

In [39]:
#hyper params
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1

#TODO try to use other model parameters

emb_dim = 256
hidden = 128
n_epochs = 8
cuda_device = 0
batch_size = 128
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

In [40]:
#nn.RNN
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes, nn.RNN).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [41]:
%%time
model_rnn_time = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')
print(datetime.datetime.now()-model_rnn_time)

epoch: 0, step: 0, loss: 3.22158145904541
epoch: 0, step: 100, loss: 0.39390280842781067
epoch: 1, step: 0, loss: 0.1897428184747696
epoch: 1, step: 100, loss: 0.23065243661403656
epoch: 2, step: 0, loss: 0.14553719758987427
epoch: 2, step: 100, loss: 0.16135287284851074
epoch: 3, step: 0, loss: 0.11805910617113113
epoch: 3, step: 100, loss: 0.1121533215045929
epoch: 4, step: 0, loss: 0.06896404922008514
epoch: 4, step: 100, loss: 0.0650561973452568
epoch: 5, step: 0, loss: 0.05571252107620239
epoch: 5, step: 100, loss: 0.07802284508943558
epoch: 6, step: 0, loss: 0.08685595542192459
epoch: 6, step: 100, loss: 0.06544125825166702
epoch: 7, step: 0, loss: 0.07677440345287323
epoch: 7, step: 100, loss: 0.055281490087509155
0:00:09.200360
CPU times: user 8.61 s, sys: 413 ms, total: 9.02 s
Wall time: 9.2 s


In [42]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(end)

['PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.000992


In [43]:
#nn.GRU
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes, nn.GRU).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [44]:
%%time
model_gru_time = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')
print(datetime.datetime.now()-model_gru_time)

epoch: 0, step: 0, loss: 3.357534646987915
epoch: 0, step: 100, loss: 0.3646502196788788
epoch: 1, step: 0, loss: 0.22849565744400024
epoch: 1, step: 100, loss: 0.14016509056091309
epoch: 2, step: 0, loss: 0.16559894382953644
epoch: 2, step: 100, loss: 0.11204125732183456
epoch: 3, step: 0, loss: 0.08227438479661942
epoch: 3, step: 100, loss: 0.10856962203979492
epoch: 4, step: 0, loss: 0.09864313900470734
epoch: 4, step: 100, loss: 0.08829253166913986
epoch: 5, step: 0, loss: 0.0862148255109787
epoch: 5, step: 100, loss: 0.05872223898768425
epoch: 6, step: 0, loss: 0.07429233938455582
epoch: 6, step: 100, loss: 0.05343397334218025
epoch: 7, step: 0, loss: 0.07046549767255783
epoch: 7, step: 100, loss: 0.07354164123535156
0:00:11.501236
CPU times: user 10.9 s, sys: 390 ms, total: 11.3 s
Wall time: 11.5 s


In [45]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(end)

['PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.000983


In [46]:
#nn.LSTM
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes, nn.LSTM).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [47]:
%%time
model_lstm_time = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')
print(datetime.datetime.now()-model_lstm_time)

epoch: 0, step: 0, loss: 2.725430727005005
epoch: 0, step: 100, loss: 0.5440400838851929
epoch: 1, step: 0, loss: 0.19133931398391724
epoch: 1, step: 100, loss: 0.16222286224365234
epoch: 2, step: 0, loss: 0.1892334669828415
epoch: 2, step: 100, loss: 0.08681514859199524
epoch: 3, step: 0, loss: 0.11258475482463837
epoch: 3, step: 100, loss: 0.09187579900026321
epoch: 4, step: 0, loss: 0.0657251700758934
epoch: 4, step: 100, loss: 0.07162638008594513
epoch: 5, step: 0, loss: 0.09976565092802048
epoch: 5, step: 100, loss: 0.07832629978656769
epoch: 6, step: 0, loss: 0.07866831868886948
epoch: 6, step: 100, loss: 0.050223883241415024
epoch: 7, step: 0, loss: 0.05137931555509567
epoch: 7, step: 100, loss: 0.05563878268003464
0:00:13.518751
CPU times: user 12.9 s, sys: 386 ms, total: 13.3 s
Wall time: 13.5 s


In [48]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(end)

['PRON', 'VERB', 'ADV', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.001727


In [49]:
#2 часть

In [6]:
def collate_fn(input_data):
    data = []
    chars = []
    targets = []
    max_len = 0
    for item in input_data:
        if len(item['data']) > max_len:
            max_len = len(item['data'])
        data.append(torch.as_tensor(item['data']))
        chars.append(item['char'])
        targets.append(torch.as_tensor(item['target']))
    chars_seq = [[torch.as_tensor([0]) for _ in range(len(input_data))] for _ in range(max_len)]
    for j in range(len(input_data)):
        for i in range(max_len):
            if len(chars[j]) > i:
                chars_seq[i][j] = torch.as_tensor(chars[j][i])
    for j in range(max_len):
        chars_seq[j] = pad_sequence(chars_seq[j], batch_first=True, padding_value=0)
    data = pad_sequence(data, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return {'data': data, 'chars': chars_seq, 'target': targets}

In [7]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.char_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        emb = self.char_emb(x) # B x T x Emb_dim
        _, out = self.rnn(emb)
        # _: B x T x Hidden 
        # out: 1 x B x Hidden

        return out.transpose(0, 1) # B x 1 x Hidden

In [8]:
#hyper params
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1

#TODO try to use other model parameters
emb_dim = 256
hidden = 128
char_hid = 128
char_emb = 64
n_epochs = 8
cuda_device = 0
batch_size = 128
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

In [9]:
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden, class_model):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.class_model = class_model(emb_dim + char_hidden, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim
        self.char_rnn = CharRNN(char_vocab, char_emb, char_hidden)

    def forward(self, x, chars):
        emb = self.word_emb(x)
        char_features = [self.char_rnn(c.to(x.device)) for c in chars]
        char_features = torch.cat(char_features, dim=1) # конкатенация по времени B x T x Char_hid
        emb = torch.cat((emb, char_features), dim=-1) # конкатенация векторов
        hidden, _ = self.class_model(emb)
        pred = self.clf(self.do(hidden))

        return pred

In [54]:
#nn.GRU
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid, nn.GRU).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [55]:
%%time
model_GRU_time = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device), batch['chars'])
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')
print(datetime.datetime.now()-model_GRU_time)

epoch: 0, step: 0, loss: 2.51133394241333
epoch: 0, step: 100, loss: 0.23598967492580414
epoch: 1, step: 0, loss: 0.20292165875434875
epoch: 1, step: 100, loss: 0.10102644562721252
epoch: 2, step: 0, loss: 0.09344164282083511
epoch: 2, step: 100, loss: 0.06658626347780228
epoch: 3, step: 0, loss: 0.0903792530298233
epoch: 3, step: 100, loss: 0.06118911877274513
epoch: 4, step: 0, loss: 0.06782786548137665
epoch: 4, step: 100, loss: 0.06826453655958176
epoch: 5, step: 0, loss: 0.06583750247955322
epoch: 5, step: 100, loss: 0.04947122558951378
epoch: 6, step: 0, loss: 0.05924211069941521
epoch: 6, step: 100, loss: 0.05668460205197334
epoch: 7, step: 0, loss: 0.04545280337333679
epoch: 7, step: 100, loss: 0.056430328637361526
0:03:37.449731
CPU times: user 3min 36s, sys: 3.77 s, total: 3min 39s
Wall time: 3min 37s


In [56]:
#example
#TODO modify inference for model with char input
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(end)

['PRON', 'VERB', 'ADV', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.003065


In [10]:
#nn.RNN
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid, nn.RNN).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [11]:
%%time
model_RNN_time = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device), batch['chars'])
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')
print(datetime.datetime.now()-model_RNN_time)

epoch: 0, step: 0, loss: 3.0529861450195312
epoch: 0, step: 100, loss: 0.21675074100494385
epoch: 1, step: 0, loss: 0.21850775182247162
epoch: 1, step: 100, loss: 0.17626258730888367
epoch: 2, step: 0, loss: 0.11362176388502121
epoch: 2, step: 100, loss: 0.09027150273323059
epoch: 3, step: 0, loss: 0.09947803616523743
epoch: 3, step: 100, loss: 0.07473014295101166
epoch: 4, step: 0, loss: 0.06810516864061356
epoch: 4, step: 100, loss: 0.046909499913454056
epoch: 5, step: 0, loss: 0.04895896464586258
epoch: 5, step: 100, loss: 0.045384228229522705
epoch: 6, step: 0, loss: 0.04080502316355705
epoch: 6, step: 100, loss: 0.051320333033800125
epoch: 7, step: 0, loss: 0.05082635208964348
epoch: 7, step: 100, loss: 0.05180642753839493
0:03:50.596601
CPU times: user 3min 44s, sys: 4.5 s, total: 3min 49s
Wall time: 3min 50s


In [12]:
#example
#TODO modify inference for model with char input
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(end)

['PRON', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.007088


In [13]:
#nn.LSTM
model = RNNPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid, nn.LSTM).to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [14]:
%%time
model_LSTM_time = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device), batch['chars'])
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(model.state_dict(), f'./rnn_chkpt_{epoch}.pth')
print(datetime.datetime.now()-model_LSTM_time)

epoch: 0, step: 0, loss: 2.8396124839782715
epoch: 0, step: 100, loss: 0.3641582131385803
epoch: 1, step: 0, loss: 0.14743974804878235
epoch: 1, step: 100, loss: 0.13808219134807587
epoch: 2, step: 0, loss: 0.10868661850690842
epoch: 2, step: 100, loss: 0.12357311695814133
epoch: 3, step: 0, loss: 0.06724485754966736
epoch: 3, step: 100, loss: 0.08600334823131561
epoch: 4, step: 0, loss: 0.06318327784538269
epoch: 4, step: 100, loss: 0.06649371981620789
epoch: 5, step: 0, loss: 0.056914668530225754
epoch: 5, step: 100, loss: 0.05416404455900192
epoch: 6, step: 0, loss: 0.029226042330265045
epoch: 6, step: 100, loss: 0.05820460990071297
epoch: 7, step: 0, loss: 0.05282336845993996
epoch: 7, step: 100, loss: 0.040207624435424805
0:04:21.838769
CPU times: user 3min 54s, sys: 3.85 s, total: 3min 58s
Wall time: 4min 21s


In [15]:
#example
#TODO modify inference for model with char input
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model.eval()
    predict = model(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(end)

['PRON', 'VERB', 'ADV', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'PRON']
0:00:00.009018


In [18]:
import pandas as pd
itog = pd.DataFrame(data=[['9.20', '11.50', '13.54', '3.37.44', '3.50.59', '4.21.83'],
                            ['0.00099', '0.00098', '0.00172', '0.00301','0.0070','0.0090']],
                      index = ['Обучение','Тест'],
                      columns = ['RNN', 'GRU', 'LSTM','RNN_char', 'GRU_char', 'LSTM_char'])
itog

,RNN,GRU,LSTM,RNN_char,GRU_char,LSTM_char
Обучение,9.20,11.50,13.54,3.37.44,3.50.59,4.21.83
Тест,0.00099,0.00098,0.00172,0.00301,0.0070,0.0090
